In [1]:
"""
init
"""
import os
import io
import re
import ast
import pdb
import clip
import json
import torch
import faiss
import chromadb
import numpy as np
from PIL import Image
from tqdm import tqdm
Image.MAX_IMAGE_PIXELS = None
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoTokenizer
from transformers import CLIPProcessor, CLIPModel

# init clip
clip_model, preprocess = clip.load("ViT-B/16", device="cuda")

# save index
def index_save(index_split, path):
    faiss.write_index(index_split, path)
    return

[2024-03-07 00:49:02,902] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [4]:
"""
build memory
"""
file_path = "/mnt/petrelfs/liuziyu/LLM_Memory/SimplyRetrieve/classification/New_test/database/cub200/cub200_database.txt"
with open(file_path, 'r') as file:
    lines = file.readlines()
image_paths = [line.split()[0] for line in lines]
print(len(image_paths))

### index保存位置
index_img_save_path =  "/mnt/petrelfs/liuziyu/LLM_Memory/SimplyRetrieve/classification/New_test/database/cub200/cub200_database.index"

### 建立一个图片index
index_img = faiss.IndexHNSWFlat(512, 64, faiss.METRIC_INNER_PRODUCT)

### 图片向量化
embed_img = []
with torch.no_grad():
    for image_path in tqdm(image_paths,desc="Process:"):
        image = preprocess(Image.open(image_path)).unsqueeze(0).to("cuda")
        image_features = clip_model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        embed_img.append(image_features.cpu())
embed_img = [np.array(embed) for embed in embed_img]
embed_img = np.array(embed_img).squeeze()
index_img.add(embed_img)
print("Total number of indexes:", index_img.ntotal)

### 保存index
print("Saving index:")
index_save(index_img, index_img_save_path)
print("Done")

600


Process:: 100%|██████████| 600/600 [00:07<00:00, 77.69it/s]

Total number of indexes: 600
Saving index:
Done


In [ ]:
index_img_save_path =  "/mnt/petrelfs/liuziyu/LLM_Memory/SimplyRetrieve/classification/New_test/database/pets37/pets37_database.index"
trainset_file_path = "/mnt/petrelfs/liuziyu/LLM_Memory/SimplyRetrieve/classification/New_test/database/pets37/pets37_database.txt"
predictions_save_path = "/mnt/petrelfs/liuziyu/LLM_Memory/SimplyRetrieve/CLIP-Cls/output/ZeroshotCLIP_topk/vit_b16/oxford_pets/predictions_finer_knn.pth"
index = faiss.read_index(index_img_save_path)
pth_file_path = '/mnt/petrelfs/liuziyu/LLM_Memory/SimplyRetrieve/CLIP-Cls/output/ZeroshotCLIP_topk/vit_b16/oxford_pets/predictions.pth'
predictions = torch.load(pth_file_path)

with open(trainset_file_path, 'r') as file:
    lines = file.readlines()
    for prediction in tqdm(predictions,desc="Process:"):
        ### 解析pth文件，获取图片位置和原来的预测结果
        for item in prediction.values():
            pre_class = item['pred_class']
            # print(item['label'])
        for item in prediction.keys():
            test_img_path = item
    
        with torch.no_grad():
            image = preprocess(Image.open(test_img_path)).unsqueeze(0).to("cuda")
            # torch.Size([1, 512])
            image_features = clip_model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            image_features = np.array(image_features.cpu())
            distance, index_result = index.search(image_features, 8)
            
            labels = []
            for index_number in index_result[0]:
                parts = lines[index_number].strip().split(' ', 1)
                part1, part2 = parts
                labels.append(int(part2))
            labels = torch.tensor(labels)
            # print(labels)
        
            ### 修改pth文件
            for item in prediction.values():
                item['pred_class'] = labels
        
torch.save(predictions, predictions_save_path)